In [1]:
import os
import pandas as pd
import numpy as np
import glob
from PIL import Image

import matplotlib.pyplot as plt

import json
from deepface.extendedmodels import Age, Gender, Race, Emotion
from deepface import DeepFace

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.preprocessing import StandardScaler

from tensorflow import keras
from os import path
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
import sys

import random
import pickle

In [12]:
## loading final features & embedding  
import numpy as np
from random import choice
from numpy import load
from numpy import expand_dims
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot
from sklearn.preprocessing import OneHotEncoder

# load face embeddings
facedata = np.load('./Data/Face/FV_10originalFacenet.npz')
facenet_train, facenet_test = facedata['arr_0'], facedata['arr_1']
# normalize input vectors
input_encoder = Normalizer(norm='l2')
facenet_train = input_encoder.fit_transform(facenet_train)
facenet_test = input_encoder.transform(facenet_test)
# label encode targets
trainy = image_train.label.values
testy = image_test.label.values

output_encoder = OneHotEncoder(sparse=False)

trainy = trainy.reshape(-1,1)
testy = testy.reshape(-1,1)

trainy = output_encoder.fit_transform(trainy)
testy = output_encoder.transform(testy)
#load voice embeddings
voicedata = np.load('./Data/Voice/FV_10originalvoice.npz')
voice_train, voice_test = voicedata['arr_0'], voicedata['arr_1']

# load demo features
demodata = np.load('./Data/Demography/FV_10originaldemo.npz')
demo_train, demo_test = demodata['arr_0'], demodata['arr_1']

facenet_train.shape, voice_train.shape, demo_train.shape, trainy.shape, facenet_test.shape, voice_test.shape,demo_test.shape, testy.shape

((229557, 128),
 (229557, 193),
 (229557, 3),
 (229557, 10),
 (7256, 128),
 (7256, 193),
 (7256, 3),
 (7256, 10))

In [13]:
import os
import pandas as pd
import numpy as np
import glob
from PIL import Image

import matplotlib.pyplot as plt

import json
from deepface.extendedmodels import Age, Gender, Race, Emotion
from deepface import DeepFace

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow import keras
from os import path
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
import sys

In [18]:
SHUFFLE_BUFFER = 1 # set this externally to the number of items in dataset for ideal results
NUM_CLASSES = 10 ####change your classes num
BATCH_SIZE = 4
LEARN_RATE = 0.01 * (BATCH_SIZE / 128)
MOMENTUM = 0.9
EPOCHS = 20
embedding_size = 128

In [22]:
from keras.models import Model
import keras.backend as K
from keras.layers import Input, Reshape, Dense, Embedding, Dropout, LSTM, Lambda, Concatenate, \
    Multiply, RepeatVector, Permute, Flatten, Activation

def Multimodal(num_classes=NUM_CLASSES, learn_rate=LEARN_RATE, momentum=MOMENTUM):
    

    inputs_facefeature = Input(shape=(128,))
    inputs_voice = Input(shape=(193,))
    inputs_demo = Input(shape=(3,))
    
### image
    wt_init = keras.initializers.RandomNormal(mean=0, stddev=0.01)
    bias_init = keras.initializers.Constant(value=0.5)
    
    def dense_layer(**args):
        return keras.layers.Dense(**args, 
            kernel_initializer=wt_init, 
            bias_initializer=bias_init)
    
### voice
    V1 = Dense(193, activation='relu')(inputs_voice)
    D1 = keras.layers.Dropout(rate=0.1, name='D1')(V1)
    V2 = Dense(128, activation='relu')(D1)
    D2 = keras.layers.Dropout(rate=0.25, name='D2')(V2)
    V3 = Dense(128, activation='relu')(D1)
    D3 = keras.layers.Dropout(rate=0.5, name='D3')(V3)

### demo    
#     # demoFeature layer 추가
    demoFeature = Dense(embedding_size, activation='tanh')(inputs_demo)
    
    # demoFeature 랑 face feature similarity 계산
    sim = Multiply()([inputs_facefeature, demoFeature])
    att = Dense(1, activation='tanh')(sim)
    att = Flatten()(att)
    
    # compute the weights of eache demographic feature
    attention = Activation('softmax')(att)
    attention = RepeatVector(embedding_size)(attention) #embedding_size
    
    # similarity랑 weights 곱하기(weighted sum)
    influence = Multiply()([sim, attention])
    influence = Lambda(lambda x: K.sum(x, axis=1))(influence)
    
    # the most important demographic feature of classification face
    influence = Dense(embedding_size)(influence)
    
    h = Concatenate()([inputs_facefeature, D3, influence])

    Softmax = dense_layer(units=num_classes, activation=keras.activations.softmax, name='F8')(h)

    model = Model(inputs=[inputs_facefeature, inputs_voice, inputs_demo],
                      outputs=[Softmax])

    sgd_opt = keras.optimizers.SGD(learning_rate=LEARN_RATE, momentum=MOMENTUM)
    cce_loss = keras.losses.categorical_crossentropy
    
    model.compile(optimizer=sgd_opt, loss=cce_loss, metrics=['accuracy'])
    
    return model

In [25]:
model1 = Multimodal()

es = EarlyStopping(monitor='loss', verbose=1, min_delta=0, mode='auto', patience= 5)
mc = ModelCheckpoint('./10originaldemo_FV.h5', monitor='loss', save_best_only=True)

### face, voice
# model1.fit([facenet_train,voice_train],trainy, steps_per_epoch = facenet_train.shape[0] // BATCH_SIZE, \
#            epochs = EPOCHS, callbacks = [es,mc])

#### face, voice, demo
model1.fit([facenet_train,voice_train, demo_train],trainy, steps_per_epoch = facenet_train.shape[0] // BATCH_SIZE, \
           epochs = EPOCHS, callbacks = [es,mc])

Epoch 1/20
57389/57389 [==============================] - 48s 839us/step - loss: 0.6590 - accuracy: 0.7982
Epoch 2/20
57389/57389 [==============================] - 48s 838us/step - loss: 0.2804 - accuracy: 0.9110
Epoch 3/20
57389/57389 [==============================] - 48s 839us/step - loss: 0.2417 - accuracy: 0.9252
Epoch 4/20
57389/57389 [==============================] - 48s 839us/step - loss: 0.1974 - accuracy: 0.9423
Epoch 5/20
57389/57389 [==============================] - 48s 837us/step - loss: 0.2051 - accuracy: 0.9397
Epoch 6/20
57389/57389 [==============================] - 48s 838us/step - loss: 0.2050 - accuracy: 0.9401
Epoch 7/20
57389/57389 [==============================] - 48s 837us/step - loss: 0.2280 - accuracy: 0.9289
Epoch 8/20
57389/57389 [==============================] - 48s 837us/step - loss: 0.2363 - accuracy: 0.9264
Epoch 9/20
57389/57389 [==============================] - 48s 839us/step - loss: 0.2084 - accuracy: 0.9403
Epoch 00009: early stopping


In [27]:
from keras.models import load_model
savedmodel = load_model('./10originaldemo_FV.h5')

### face, voice
# score = savedmodel.evaluate([facenet_test,voice_test],testy)

### face, voice, demo
score = savedmodel.evaluate([facenet_test, voice_test, demo_test],testy)

227/227 [==============================] - 0s 721us/step - loss: 0.1513 - accuracy: 0.9624
